<a href="https://colab.research.google.com/github/GreggRodgers02/Colab_notebooks/blob/main/Langchain_Lesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom OpenAI Chatbot

In [ ]:
import os
from getpass import getpass

In [ ]:
import openai

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [ ]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
client = openai.OpenAI()

In [ ]:
completion = client.chat.completions.create(model="gpt-4o-mini", messages=[
                                                                           {'role': 'user',
                                                                            'content': '''
                                                                            Can you explain briefly what a black hole is?
                                                                            '''}], max_tokens=250, temperature=0.3, seed=365, stream=True
                                            )

In [ ]:
completion

In [ ]:
for i in completion:
  print(i.choices[0].delta.content, end = "")

# Tweet Classifier HW

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            'content': '''
            You are a tweet classifier, the user will give you a tween and your job is to classify the input between three categories:
            1. Positive
            2. Negative
            3. Neutral
            '''
        },
        {
            'role': 'user',
            'content': 'This product is the worst!',
            'role': 'assistant',
            'content': 'Negative',
            'role': 'user',
            'content': 'I love this product!',
            'role': 'assistant',
            'content': 'Positive',
            'role': 'user',
            'content': 'The sale starts on Tuesday.',
            'role': 'assistant',
            'content': 'Neutral',
            'role': 'user',
            'content': 'I heard that the product is releasing within the United States.'
        }
    ]
)

In [ ]:
print(completion.choices[0].message.content)

# Langchain Few-Shot Prompting

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatMessagePromptTemplate

In [ ]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.4,
    max_tokens=100,
    model_kwargs={
        "seed": 365
    }
)

In [ ]:
response = chat.invoke(''' I've recently adopted a dog. Could you suggest some dog names? ''')

In [ ]:
print(response.content)

AI Message is use in Few-Shot Prompting, were we give the model a set of example responses to learn from.

In [ ]:
message_h_dog = HumanMessage(content="I've recently adopted a dog. Could you suggest some dog names?")

message_h_cat = HumanMessage(content="I've recently adopted a cat. Could you suggest some cat names?")

In [ ]:
message_ai_dog = AIMessage(content='''Oh, sure, because naming a dog is the most important decision you'll ever make. How about "Bark Twain" for a literary twist? Or maybe "Sir Waggington" if you want to feel fancy? If you’re feeling basic, “Buddy” works too. Just don’t
 blame me when your dog doesn’t live up to the name. Good luck!''')

message_ai_cat = AIMessage(content='''Oh, sure, because naming a cat is the most important decision you’ll ever make. How about these gems:

1. Whisker McFluffface
2. Sir Purrs-a-lot
3. Catrick Swayze
4. Furrball McGee
5. Chairman Meow

Or you could just go with “Cat” and save yourself the trouble. Good luck!''')

In [ ]:
message_h_fish = HumanMessage(content="I've recently adopted a fish. Could you suggest some fish names?")

In [ ]:
message_s = SystemMessage(content='''You are marv, a chat bot reluctantly
answers questions with sarcastic responses''')

In [ ]:
#response = chat.invoke([message_h_cat, message_s])

In [ ]:
response = chat.invoke([message_s, message_h_dog, message_ai_dog,message_h_cat, message_ai_cat, message_h_fish])

In [ ]:
print(response.content)

## Prompting Template

In [ ]:
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate, FewShotChatMessagePromptTemplate, ChatPromptTemplate

In [ ]:
TEMPLATE = '''
System:
{description}

Human:
I've recently adopted a {pet}.
Could you suggest some {pet} names?
'''

In [ ]:
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [ ]:
prompt_template

In [ ]:
prompt_value = prompt_template.invoke({
    'description':"You are marv, a chat bot reluctantly answers questions with sarcastic responses",
    'pet':"dog"
    })

In [ ]:
print(prompt_value.text)

In [ ]:
TEMPLATE_S = '{description}'
TEMPLATE_H = '''
I've recently adopted a {pet}.
Could you suggest some {pet} names?
'''

message_template_s = SystemMessagePromptTemplate.from_template(template=TEMPLATE_S)
message_template_h = HumanMessagePromptTemplate.from_template(template=TEMPLATE_H)

In [ ]:
message_template_s

In [ ]:
message_template_h

In [ ]:
chat_template = ChatPromptTemplate.from_messages([message_template_s, message_template_h])

In [ ]:
chat_template

In [ ]:
chat_value = chat_template.invoke({
    'description':"You are marv, a chat bot reluctantly answers questions with sarcastic responses",
    'pet':"dog"
})

In [ ]:
chat_value

In [ ]:
response = chat.invoke(chat_value)

In [ ]:
response.content

#Cleaner Few Shot Prompting

In [ ]:
TEMPLATE_H = '''
I've recently adopted a {pet}.
Could you suggest some {pet} names?
'''
TEMPLATE_AI = '''{response}'''

message_template_h = HumanMessagePromptTemplate.from_template(template=TEMPLATE_H)
message_template_ai = AIMessagePromptTemplate.from_template(template=TEMPLATE_AI)

In [ ]:
example_template = ChatPromptTemplate.from_messages([message_template_h, message_template_ai])

In [ ]:
examples = [
    {'pet': 'dog',
             'response':'''Oh, sure, because naming a dog is the most important decision you'll ever make. How about "Bark Twain" for a literary twist? Or maybe "Sir Waggington" if you want to feel fancy? If you’re feeling basic, “Buddy” works too. Just don’t
 blame me when your dog doesn’t live up to the name. Good luck!'''},

  {'pet': 'cat',
   'response':'''Oh, sure, because naming a cat is the most important decision you’ll ever make. How about these gems:

1. Whisker McFluffface
2. Sir Purrs-a-lot
3. Catrick Swayze
4. Furrball McGee
5. Chairman Meow

Or you could just go with “Cat” and save yourself the trouble. Good luck!'''}
    ]

In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(examples=examples,
                                                   example_prompt=example_template,
                                                   input_variables=['pet'])

In [ ]:
chat_template = ChatPromptTemplate.from_messages([few_shot_prompt, message_template_h])

In [ ]:
chat_value = chat_template.invoke({
    'pet': 'rabbit'
})

In [ ]:
for i in chat_value.messages:
  print(f"{i.type}: {i.content} \n")

In [ ]:
response = chat.invoke(chat_value)

In [ ]:
response.content

# Output Parser

The string output parser  converts an output to a string object

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
message_h = HumanMessage("Can you give me an interesting fact I probably didn't know about?")

In [ ]:
response = chat.invoke([message_h])

In [ ]:
response

In [ ]:
str_output_parser = StrOutputParser()

In [ ]:
response_parsed = str_output_parser.invoke(response)
response_parsed

Comma Parser returns a list

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [ ]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.4,
    max_tokens=100,
    model_kwargs={
        "seed": 365
    }
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


We can insert the instructions of the parser directly within the human message

In [ ]:
message_h = HumanMessage(content=f'''Ive recently adopted a dog. Could you suggest some dog names?


{CommaSeparatedListOutputParser().get_format_instructions()}


''')

In [ ]:
print(message_h.content)

Ive recently adopted a dog. Could you suggest some dog names?
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`



In [ ]:
reponse = chat.invoke([message_h])

In [ ]:
print(reponse.content)

Max, Bella, Charlie, Luna, Cooper, Daisy, Rocky, Sadie, Buddy, Molly, Duke, Zoe, Teddy, Ruby, Oliver, Chloe, Finn, Lily, Bear, Mia


In [ ]:
list_output_parser = CommaSeparatedListOutputParser()

In [ ]:
response_parsed = list_output_parser.invoke(reponse)

In [ ]:
response_parsed

['Max',
 'Bella',
 'Charlie',
 'Luna',
 'Cooper',
 'Daisy',
 'Rocky',
 'Sadie',
 'Buddy',
 'Molly',
 'Duke',
 'Zoe',
 'Teddy',
 'Ruby',
 'Oliver',
 'Chloe',
 'Finn',
 'Lily',
 'Bear',
 'Mia']

In [ ]:
list_output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

Datetime parser

In [ ]:
import langchain_core.output_parsers as op
dir(op)

['BaseCumulativeTransformOutputParser',
 'BaseGenerationOutputParser',
 'BaseLLMOutputParser',
 'BaseOutputParser',
 'BaseTransformOutputParser',
 'CommaSeparatedListOutputParser',
 'JsonOutputKeyToolsParser',
 'JsonOutputParser',
 'JsonOutputToolsParser',
 'ListOutputParser',
 'MarkdownListOutputParser',
 'NumberedListOutputParser',
 'PydanticOutputParser',
 'PydanticToolsParser',
 'SimpleJsonOutputParser',
 'StrOutputParser',
 'XMLOutputParser']

In [ ]:
from langchain_core.output_parsers import JsonOutputParser, SimpleJsonOutputParser

In [ ]:
print(SimpleJsonOutputParser().get_format_instructions())

Return a JSON object.


## **Piping a Prompt, Model, and an Output Parser**

In [ ]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [ ]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [ ]:
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ('human',
     'I have recently adopted a {pet}. Could you suggest somthreee {pet} names? \n' + list_instructions)
])

In [ ]:
print(chat_template.messages[0].prompt.template)

I have recently adopted a {pet}. Could you suggest somthreee {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=100,
    model_kwargs={"seed": 365}
)

In [ ]:
list_output_parser = CommaSeparatedListOutputParser()

In [ ]:
chat_template_results = chat_template.invoke({'pet': 'dog'})

In [ ]:
chat_results = chat.invoke(chat_template_results)

In [ ]:
list_output_parser.invoke(chat_results)

['Buddy', 'Bella', 'Max']

In [ ]:
chain = (
    chat_template
    | chat
    | list_output_parser
)

chain.invoke({
    'pet': 'cat'
    })

['Whiskers', 'Luna', 'Oliver']

## **Batching**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ('human',
     'I have recently adopted a {pet} which is a {breed}. Could you suggest several training tips?')
])

In [ ]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=100,
    model_kwargs={"seed": 365}
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
chain = (
    chat_template
    | chat
)

In [ ]:
chain.invoke({
    'pet': 'dog',
    'breed': 'golden retriever'
})

AIMessage(content='Congratulations on your new golden retriever! They are known for their friendly and intelligent nature, making them great companions. Here are several training tips to help you get started:\n\n1. **Start with Basic Commands**: Teach essential commands like "sit," "stay," "come," "down," and "leave it." Use positive reinforcement, such as treats and praise, to encourage good behavior.\n\n2. **Consistency is Key**: Use the same commands and cues consistently. This helps your dog understand', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 27, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_454234172d', 'id': 'chatcmpl-D6mbuowUB1K0EVo2w

The batch() method runs invoke() in parallel

In [ ]:
%%time
chain.batch([
    {
        'pet': 'dog',
        'breed': 'shepard'
    },
    {
        'pet': 'dragon',
        'breed': 'night fury'
    }
    ])

CPU times: user 26 ms, sys: 0 ns, total: 26 ms
Wall time: 1.25 s


[AIMessage(content='Congratulations on your new shepherd! Here are several training tips to help you and your dog build a strong bond and ensure good behavior:\n\n1. **Start with Basic Commands**: Teach essential commands like "sit," "stay," "come," "down," and "leave it." Use positive reinforcement, such as treats and praise, to encourage your dog.\n\n2. **Consistency is Key**: Use the same commands and gestures consistently. This helps your dog understand what you expect from them.\n\n3.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 26, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_454234172d', 'id': 'chatcmpl-D6mdgkexMHOdQgr6wXjjiE5qXApVx', 's

## **Streaming**

In [ ]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=100,
    model_kwargs={"seed": 365}
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
chain = (
    chat_template
    | chat
)

In [ ]:
response = chain.stream({
    'pet': 'dog',
    'breed': 'Boxer'
})

Generator Functions allow for defining functions that behave like iterators, allowing us to loop over there output

In [ ]:
next(response)

AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--019c3a9e-0a7c-7d21-8140-0dbcb7b4e528', tool_calls=[], invalid_tool_calls=[], tool_call_chunks=[])

To access the elements in a generator one at atime, we set it as an argument to the built in python function called next

In [ ]:
for i in response:
  print(i.content, end = "")

Congratulations on adopting a Boxer! They are known for their playful and energetic nature, so training is essential for a well-behaved companion. Here are several training tips to help you get started:

1. **Start with Basic Commands**: Teach basic commands like "sit," "stay," "come," and "down." Use positive reinforcement, such as treats and praise, to encourage good behavior.

2. **Consistency is Key**: Be consistent with commands and rules. Use the same words and

The end argument allow us to display the results horizontally

## **The Runnable and RunnableSequence Classes**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
chat_template = ChatPromptTemplate.from_messages([
    ('human',
     'I have recently adopted a {pet} which is a {breed}. Could you suggest several training tips?')
])

In [ ]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=100,
    model_kwargs={"seed": 365}
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
chain = chat_template | chat

In [ ]:
type(chat_template)

langchain_core.prompts.chat.ChatPromptTemplate

A runnable is a unit of work that can be invoked, batched, streamed, transformed and composed

## **Piping Chains and the RunnablePassthrough Class**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
RunnablePassthrough().invoke('Hello')

'Hello'

In [ ]:
chat_template_tools = ChatPromptTemplate.from_template(
    '''
    What are the five most important tools a {job_title} needs?
    Answer only by listing the tools.
    '''
)

chat_template_strategy = ChatPromptTemplate.from_template(
    '''
    Considering the tools provided, develop a strategy for effectively learning and mastering them:
    {tools}
    '''
)

In [ ]:
chat_template_tools

ChatPromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job_title'], input_types={}, partial_variables={}, template='\n    What are the five most important tools a {job_title} needs?\n    Answer only by listing the tools.\n    '), additional_kwargs={})])

In [ ]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=100,
    model_kwargs={"seed": 365}
)

In [ ]:
string_parser = StrOutputParser()

In [ ]:
chain_tools = (
    chat_template_tools
    | chat
    | string_parser
    | {'tools': RunnablePassthrough()}
)

chain_strategy = (
    chat_template_strategy
    | chat
    | string_parser
)

In [ ]:
tools_results = chain_tools.invoke({'job_title': 'data scientist'})

In [ ]:
print(tools_results)

{'tools': '1. Python\n2. R\n3. SQL\n4. Jupyter Notebook\n5. TensorFlow/PyTorch'}


In [ ]:
print(chain_strategy.invoke({
    'tools': tools_results
    }))

To effectively learn and master the tools you've listed—Python, R, SQL, Jupyter Notebook, and TensorFlow/PyTorch—it's essential to develop a structured strategy that incorporates both theoretical understanding and practical application. Here’s a step-by-step approach:

### 1. **Set Clear Goals**
   - Define what you want to achieve with each tool. For example:
     - Python: General programming and data analysis.
     - R: Statistical analysis and data visualization.
     - SQL: Database


In [ ]:
chain_combined = (
    chain_tools
    | chain_strategy
)

In [ ]:
chain_combined.invoke({'job_title': 'data scientist'})

"To effectively learn and master the tools you've listed—Python, R, SQL, Jupyter Notebook, and TensorFlow/PyTorch—it's essential to develop a structured strategy that incorporates both theoretical understanding and practical application. Here’s a step-by-step approach:\n\n### 1. **Set Clear Goals**\n   - Define what you want to achieve with each tool. For example:\n     - Python: General programming and data analysis.\n     - R: Statistical analysis and data visualization.\n     - SQL: Database"

In [ ]:
Long_chain = (
    chat_template_tools
    | chat
    | string_parser
    | {'tools': RunnablePassthrough()}
    | chat_template_strategy
    | chat
    | string_parser
)

## **Graphing Runnables**

In [ ]:
#!pip install grandalf

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
chat_template_tools = ChatPromptTemplate.from_template(
    '''
    What are the five most important tools a {job_title} needs?
    Answer only by listing the tools.
    '''
)

chat_template_strategy = ChatPromptTemplate.from_template(
    '''
    Considering the tools provided, develop a strategy for effectively learning and mastering them:
    {tools}
    '''
)

In [ ]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=100,
    model_kwargs={"seed": 365}
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
string_parser = StrOutputParser()

In [ ]:
chain_long = (
    chat_template_tools
    | chat
    | string_parser
    | {'tools': RunnablePassthrough()}
    | chat_template_strategy
    | chat
    | string_parser
)

In [ ]:
chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

## **RunnableParallel**

In [ ]:
# !pip install langchain_openai
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [ ]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books.
    Answer only bu listing the books
    '''
)

chat_template_project = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers.
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
    '''
    I am an intermeidiate level programmer.

    Consider the following literature:
    {books}

    Also, consider the following projects:
    {projects}

    Roughly how much time would it take me to complete the literature and the projects?

    '''
)

In [ ]:
chat = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=500,
    model_kwargs={"seed": 365}
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
string_parser = StrOutputParser()

In [ ]:
chain_books = (
    chat_template_books
    | chat
    | string_parser
)

chain_projects = (
    chat_template_project
    | chat
    | string_parser
)

In [ ]:
chain_parallel = RunnableParallel(
    {'books': chain_books, 'projects': chain_projects}
)

In [ ]:
chain_parallel.invoke({'programming language': 'python'})

{'books': '1. "Fluent Python" by Luciano Ramalho  \n2. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin  \n3. "Python Cookbook" by David Beazley and Brian K. Jones  ',
 'projects': '1. Personal Finance Tracker with Data Visualization  \n2. Web Scraper with Automated Reporting  \n3. Chatbot using Natural Language Processing'}

In [ ]:
chain_time_1 = (
    RunnableParallel({'books': chain_books, 'projects': chain_projects})
    | chat_template_time
    | chat
    | string_parser
)


#Best practice
chain_time_2 = (
     ({'books': chain_books,
       'projects': chain_projects})
    | chat_template_time
    | chat
    | string_parser
)

In [ ]:
print(chain_time_2.invoke({'programming language': 'python'}))

The time it takes to complete the literature and projects can vary significantly based on your current skill level, the depth of understanding you aim to achieve, and the complexity of the projects. However, I can provide a rough estimate for each component.

### Literature

1. **"Fluent Python" by Luciano Ramalho**  
   - Estimated Time: 4-6 weeks  
   - This book is comprehensive and covers advanced Python concepts. If you dedicate around 10-15 hours per week, you can expect to finish it in about a month.

2. **"Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin**  
   - Estimated Time: 2-3 weeks  
   - This book is more concise and can be read more quickly. Spending about 5-10 hours per week should allow you to complete it in a couple of weeks.

3. **"Python Cookbook" by David Beazley and Brian K. Jones**  
   - Estimated Time: 3-4 weeks  
   - This book is practical and involves working through recipes. If you spend around 10 hours a week, you can finish it

## **Runnable Lambda**

In [ ]:
from langchain_core.runnables import RunnableLambda

In [ ]:
find_sum = lambda x: sum(x)

In [ ]:
find_sum([1, 2, 5])

8

In [ ]:
find_square = lambda x: x**2

In [ ]:
find_square(4)

16

In [ ]:
runnable_sum = RunnableLambda(lambda x: sum(x))
runnable_square = RunnableLambda(lambda x: x**2)

In [ ]:
runnable_sum.invoke([1, 2, 5])

8

In [ ]:
runnable_square.invoke(4)

16

In [ ]:
chain = (
    runnable_sum
    | runnable_square
)

In [ ]:
chain.invoke([1, 2, 5])

64

## **@chain Decorator**

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import chain

In [ ]:
def find_sum(x):
  return sum(x)

def find_square(x):
  return x**2

In [ ]:
chain1 = (
    RunnableLambda(find_sum)
    | RunnableLambda(find_square)
)

In [ ]:
chain1.invoke([1,2,5])

64

In [ ]:
@chain
def runnable_sum(x):
  return sum(x)

@chain
def runnable_square(x):
  return x**2

In [ ]:
type(runnable_sum), type(runnable_square)

(langchain_core.runnables.base.RunnableLambda,
 langchain_core.runnables.base.RunnableLambda)

In [ ]:
chain2 = (
    runnable_sum
    | runnable_square
)

In [ ]:
chain2.invoke([1,2,5])

64